In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datafile_v2.csv')

In [3]:
extra_data = pd.read_csv('datafile_v4.csv')

In [4]:
trunc_data = extra_data[['studyid', 'mrn']]

In [ ]:
trunc_data.info()

In [ ]:
df.info()

In [6]:
merge_df = pd.merge(df, trunc_data, on='studyid')


In [ ]:
merge_df.info()

In [20]:
#### Experiment 1A  (predict rvedvi_hi from ASE guideline dataset)#####

inputs_1a = ['rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvmidenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvendsystolicareacm2ap4',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'tvannulusdimensionindiastole',
'bsaonmri']

In [8]:
#alter data

# alter pulmonaryoutflowtypeattimeo and degreeoftrbyechoreportif, get rid of sex
df_2 = merge_df
#pulm outflow - want is to be native (1), trans annular patch (2), and other (rest of #s)
new_pulm_col = []
for i in df_2['pulmonaryoutflowtypeattimeo']:
    if i == 1 or i == 2:
        new_pulm_col.append(i)
    else:
        new_pulm_col.append(3)
        
df_2['new_pulm_outflow_col'] = new_pulm_col

#tr - greater than mild (3+) vs not (0,1,2)
new_tr_col = []
for i in df_2['degreeoftrbyechoreportif']:
    if i == 1 or i == 2 or i == 0:
        new_tr_col.append(0)
    else:
        new_tr_col.append(1)
        
df_2['tr_greater_than_mild'] = new_tr_col

df_2 = df_2.drop(columns=['sex', 'pulmonaryoutflowtypeattimeo', 'degreeoftrbyechoreportif'])


In [ ]:
df_2['tr_greater_than_mild'].value_counts()

In [22]:
#save all 1A stuff
average_performance_1A = average_performance
std_dev_performance_1A = std_dev_performance
mean_tpr_1A = mean_tpr
std_tpr_1A = std_tpr
mean_fprs_1A = mean_fprs

In [ ]:
### xgboost auroc with group stratification and cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# data setup
X = merge_df[inputs_1a]
y = merge_df['outcome_rvedvi_hi']
groups = merge_df['mrn']  # Group identifier

# Hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Number of splits for cross-validation
n_splits_outer = 5

# GroupKFold for outer cross-validation
outer_cv = GroupKFold(n_splits=n_splits_outer)

# Lists for storing results
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)
tpr_interpolations = []

#store feature importance per fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # RandomizedSearchCV for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='roc_auc',
        cv=3,  # You can adjust this value
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Average auroc across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', lw=2, label=f'Mean ROC (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='grey', alpha=0.2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#xgboost auroc with nested cross validation 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df[inputs_1a]
y = df['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

#make list to store feature importances for each fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)
    
    #save feature importances for this fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(temp_fi)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Plotting the ROC curves
plt.figure(figsize=(7, 7))
plt.plot(mean_fprs_1A, mean_tpr_1A, color='b', lw=2, label=f'Mean ROC for Run 1A (AUC = {average_performance_1A:.2f})')
plt.fill_between(mean_fprs_1A, mean_tpr_1A - std_tpr_1A, mean_tpr_1A + std_tpr_1A, color='grey', alpha=0.15)

plt.plot(mean_fprs_2A4, mean_tpr_2A4, color='g', lw=2, label=f'Mean ROC for Run 2A v.4 (AUC = {average_performance_2A4:.2f})')
plt.fill_between(mean_fprs_2A4, mean_tpr_2A4 - std_tpr_2A4, mean_tpr_2A4 + std_tpr_2A4, color='grey', alpha=0.15)

plt.plot(mean_fprs_2A7, mean_tpr_2A7, color='r', lw=2, label=f'Mean ROC for Run 2A v.7 (AUC = {average_performance_2A7:.2f})')
plt.fill_between(mean_fprs_2A7, mean_tpr_2A7 - std_tpr_2A7, mean_tpr_2A7 + std_tpr_2A7, color='grey', alpha=0.15)

plt.plot([], [], color='grey', alpha=0.2, linewidth=10, label='±1 SD')


plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')

plt.savefig("roc_curve_rvedv.svg", format='svg')

plt.show()


In [ ]:
#xgboost auroc with REPEATED (20x) nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# setup
X = df[inputs_1a]
y = df['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5
n_repeats_outer = 20

# make RepeatedStratifiedKFold cross-validation iterator for outer CV
outer_cv = RepeatedStratifiedKFold(n_splits=n_splits_outer, n_repeats=n_repeats_outer, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

#create list to store feature importances for each fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)





In [ ]:
# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')

plt.savefig('1a_auroc_graph.svg', format='svg')

plt.show()

In [40]:
#get mean and std dev of feature importances 

col0 = []
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []
col7 = []
col8 = []



for f in feature_importances:
    
    count = 0
    for i in f:
        if count == 0:
            col0.append(i)
        if count == 1:
            col1.append(i)
        if count == 2:
            col2.append(i)
        if count == 3:
            col3.append(i)
        if count == 4:
            col4.append(i)
        if count == 5:
            col5.append(i)
        if count == 6:
            col6.append(i)
        if count == 7:
            col7.append(i)
        if count == 8:
            col8.append(i)
            
        count = count + 1
            
            


In [ ]:
print(np.mean(col8))
print(np.std(col8))

In [ ]:
#make bar graph plot for aurocs
# AUROC scores and standard deviations
auroc_scores = [average_performance_1A, average_performance_2A4, average_performance_2A7]
std_devs = [std_dev_performance_1A, std_dev_performance_2A4, std_dev_performance_2A7]

# X-axis labels
labels = ['Model 1', 'Model 2A v.4', 'Model 2A v.7']

# Creating the bar plot
plt.figure(figsize=(7, 7))
plt.bar(labels, auroc_scores, yerr=std_devs, capsize=5, color='skyblue')

# Adding titles and labels
plt.ylabel('AUROC Score')
plt.title('AUROC Scores with Standard Deviation')
plt.ylim([0, 1])  # Optional: Ensuring y-axis starts at 0 and ends at 1
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Optional: Adding a grid for readability

plt.savefig("roc_barplot_rvedv.svg", format='svg')


# Show the plot
plt.show()

In [ ]:
#get feature importances for each fold

#print(feature_importances)

for i in X_train.columns:
    print(i)

In [ ]:
## get feature importance

feature_importance = best_xgb_model.feature_importances_
feature_names = X_train.columns
# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, tick_label=feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('XGBoost Classifier Feature Importance')
plt.show()

In [ ]:
print(sum(feature_importance))

In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df[inputs_1a]
y = df['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 1A)')
plt.show()


In [ ]:
#xgboost pr curves with group strat + nested cv loops

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# setup
X = merge_df[inputs_1a]
y = merge_df['outcome_rvedvi_hi']
groups = merge_df['mrn']  # Group identifier column

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# Number of splits for outer and inner cross-validation
n_splits_outer = 5
n_splits_inner = 3

# GroupKFold for outer cross-validation
outer_group_kfold = GroupKFold(n_splits=n_splits_outer)

# Lists to store results
precisions_list = []
recalls_list = []
average_aucpr_scores = []

# Outer cross-validation loop
for train_index, test_index in outer_group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    groups_train = groups.iloc[train_index]

    # XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # GroupKFold for inner cross-validation (hyperparameter tuning)
    inner_group_kfold = GroupKFold(n_splits=n_splits_inner)

    # RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='average_precision',
        cv=inner_group_kfold.split(X_train, y_train, groups_train),
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    # PR curve values
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    precisions_list.append(precision)
    recalls_list.append(recall)

    # AUC-PR
    auc_pr = auc(recall, precision)
    average_aucpr_scores.append(auc_pr)

# Calculate and print the average and standard deviation performance across all outer folds
average_aucpr = np.mean(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", average_aucpr)
std_aucpr = np.std(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", std_aucpr)

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))
mean_recall = np.linspace(0, 1, 100)
precision_interp = []

# Interpolate and plot each fold's precision-recall curve
for recall, precision in zip(recalls_list, precisions_list):
    recall_sorted, precision_sorted = zip(*sorted(zip(recall, precision)))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')

ax.fill_between(mean_recall, mean_precision - std_aucpr, mean_precision + std_aucpr, color='blue', alpha=0.2)
plt.plot([], [], color='grey', alpha=0.2, linewidth=10, label='±1 SD')


plt.subplots_adjust(right=0.75)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds (Experiment 1A)')
plt.show()


In [38]:
#save 1A stuff

mean_precision_1A = mean_precision
mean_recall_1A = mean_recall
average_aucpr_1A = average_aucpr
std_aucpr_1A = std_aucpr
baseline_prevalence_1A = baseline_prevalence

In [ ]:
#overlaying plots
# Plotting
fig, ax = plt.subplots(figsize=(7, 7))

ax.plot(mean_recall_1A, mean_precision_1A, color='b', linestyle='-', lw=2, label=f'Mean AUPRC for Run 1A (AUC = {round(average_aucpr_1A, 3)})')

ax.hlines(baseline_prevalence_1A, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence_1A, 3)})')

ax.fill_between(mean_recall_1A, mean_precision_1A - std_aucpr_1A, mean_precision_1A + std_aucpr_1A, color='gray', alpha=0.15)


ax.plot(mean_recall_2A4, mean_precision_2A4, color='g', linestyle='-', lw=2, label=f'Mean AUPRC for Run 2A v.4 (AUC = {round(average_aucpr_2A4, 3)})')

#ax.hlines(baseline_prevalence_2A4, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence_2A4, 3)})')

ax.fill_between(mean_recall_2A4, mean_precision_2A4 - std_aucpr_2A4, mean_precision_2A4 + std_aucpr_2A4, color='gray', alpha=0.15)


ax.plot(mean_recall_2A7, mean_precision_2A7, color='r', linestyle='-', lw=2, label=f'Mean AUPRC for Run 2A v.7 (AUC = {round(average_aucpr_2A7, 3)})')

#ax.hlines(baseline_prevalence_2A7, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence_2A7, 3)})')

ax.fill_between(mean_recall_2A7, mean_precision_2A7 - std_aucpr_2A7, mean_precision_2A7 + std_aucpr_2A7, color='gray', alpha=0.15)


plt.plot([], [], color='grey', alpha=0.2, linewidth=10, label='±1 SD')

plt.ylim([0, 1])  # Setting y-axis limits from 0 to 1

plt.subplots_adjust(right=0.75)
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds')

plt.savefig("pr_curve_with_std.svg", format='svg')

plt.show()

In [ ]:
#bar plot for pr 

#make bar graph plot for aurocs
# AUROC scores and standard deviations
auroc_scores = [average_aucpr_1A, average_aucpr_2A4, average_aucpr_2A7]
std_devs = [std_aucpr_1A, std_aucpr_2A4, std_aucpr_2A7]

# X-axis labels
labels = ['Model 1', 'Model 2A v.4', 'Model 2A v.7']

# Creating the bar plot
plt.figure(figsize=(7, 7))
plt.bar(labels, auroc_scores, yerr=std_devs, capsize=5, color='skyblue')

# Adding titles and labels
plt.ylabel('AUPRC Score')
plt.title('AUPRC Scores with Standard Deviation')
plt.ylim([0, 1])  # Optional: Ensuring y-axis starts at 0 and ends at 1
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Optional: Adding a grid for readability

plt.savefig("pr_barplot_rvedv.svg", format='svg')


# Show the plot
plt.show()

In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df[inputs_1a]
y = df['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [ ]:
#pr curves using REPEATED nested cv

from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df[inputs_1a]
y = df['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits and repeats for outer cross-validation
n_splits_outer = 5  # Number of outer splits
n_repeats_outer = 20  # Number of repeats

# make RepeatedStratifiedKFold cross-validation iterator for outer cross-validation
outer_repeated_stratified_kfold = RepeatedStratifiedKFold(n_splits=n_splits_outer, n_repeats=n_repeats_outer, random_state=42)

# Lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

outer_fold_auprcs = []

# Outer Cross-Validation loop
for i, (train_index, test_index) in enumerate(outer_repeated_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # add auc_pr to list
    outer_fold_auprcs.append(auc_pr)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {i+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
average_aucpr = np.mean(outer_fold_auprcs)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(outer_fold_auprcs)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [ ]:
# plot mean PR curve

plt.rcParams['font.sans-serif'] = 'Arial'

plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation')
plt.legend()

plt.savefig('1a_auprc_graph.svg', format='svg')

plt.show()

In [8]:
##### Experiment 1B ########
inputs_1b = inputs_1a

In [9]:
df_1b = df

In [10]:
df_1b = df_1b.dropna(subset=['outcome_rvesv_hi'])


In [ ]:
df_1b

In [ ]:
### xgboost auroc curves 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


# setup
X = df_1b[inputs_1b]
y = df_1b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store fpr and tpr for each fold
fprs = []
tprs = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='roc_auc',  # making AUROC our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
plt.figure(figsize=(10, 6))

# do stratification by outcome

scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    auroc_fold_scores = []

    # Fit model
    best_xgb_model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # Calculate AUROC for the current fold
    auroc = roc_auc_score(y_test, y_pred_prob)

    # Add to AUROC, TPR, FPR, and thresholds fold scores
    auroc_fold_scores.append(auroc)

    # Compute ROC curve for the current fold
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Append fpr and tpr to the lists
    fprs.append(fpr)
    tprs.append(tpr)

    # Plot the ROC curve for the current cross-validation fold
    plt.step(fprs[fold], tprs[fold], lw=1, label=f'Fold {fold+1} (AUC = {round(auroc, 3)})')

    # Add to AUROC list
    scores.append(auroc)
    
# calculate average + std AUROC score across all stratified folds
average_auroc = np.mean(scores)
print("Average AUROC Across Stratified Folds:", average_auroc)

std_auroc = np.std(scores)
print("Standard Deviation AUROC Across Stratified Folds:", std_auroc)

# Print AUROC scores for each stratified fold
for i, auroc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUROC: {auroc:.4f}")

# Calculate the mean ROC curve
mean_fpr = np.linspace(0, 1, 100)  # Choose the number of thresholds, here chose 100
tpr_interp = []

for fold in range(len(fprs)):
    tpr_interp.append(np.interp(mean_fpr, fprs[fold], tprs[fold]))

mean_tpr = np.mean(tpr_interp, axis=0)

# plot the average ROC curve and set labels
plt.plot(mean_fpr, mean_tpr, color='b', linestyle='-', lw=2, label=f'Mean ROC (AUC = {round(average_auroc,3)})')

# other plot settings
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Stratified Cross-Validation Folds (Experiment 1B)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#xgboost auroc with nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_1b[inputs_1b]
y = df_1b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df_1b[inputs_1b]
y = df_1b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 1B)')
plt.show()


In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_1b[inputs_1b]
y = df_1b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [13]:
##### Experiment 2A #####

inputs_2a = ['age_y_at_echo',
'sex',
'pulmonaryoutflowtypeattimeo_2', 'pulmonaryoutflowtypeattimeo_1', 'pulmonaryoutflowtypeattimeo_5', 'pulmonaryoutflowtypeattimeo_3', 'pulmonaryoutflowtypeattimeo_8', 'pulmonaryoutflowtypeattimeo_7', 'pulmonaryoutflowtypeattimeo_6',
'degreeofprbyechoreportif_1', 'degreeofprbyechoreportif_2', 'degreeofprbyechoreportif_3', 'degreeofprbyechoreportif_4', 'degreeofprbyechoreportif_5',
'degreeoftrbyechoreportif_0', 'degreeoftrbyechoreportif_1', 'degreeoftrbyechoreportif_2', 'degreeoftrbyechoreportif_3', 'degreeoftrbyechoreportif_4', 'degreeoftrbyechoreportif_5', 'degreeoftrbyechoreportif_6',
'tapsecmap4cviewmmode',
'rvs1tdicmsap4cviewtd',
'tvangulartiltdegreesap4c',
'mvannulusdimensioncmap4c',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvmidenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvendsystolicareacm2ap4',
'aovalvesystolicannulardimens',
'pslaxpulmvalveannulusindias',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusindiast',
'pssapulmvalveannulusinsysto',
'mpadiastolicflowreversalpres',
'branchpaatleastonediastol',
'antegradediastolicflowateith',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri',
'bsaonmri',
'rv_fac']

In [ ]:
len(inputs_2a)

In [18]:
inputs_2a_new_run2 = ['age_y_at_echo',
'degreeofprbyechoreportif_5',
'degreeoftrbyechoreportif_0',
'tapsecmap4cviewmmode',
'rvs1tdicmsap4cviewtd',
'tvangulartiltdegreesap4c',
'mvannulusdimensioncmap4c',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvmidenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvendsystolicareacm2ap4',
'aovalvesystolicannulardimens',
'pslaxpulmvalveannulusindias',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusindiast',
'pssapulmvalveannulusinsysto',
'branchpaatleastonediastol',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri',
'rv_fac']

In [20]:
inputs_2a_new_run3 = ['weightonmri',
'heightonmri',
'rvendsystolicareacm2ap4',
'aovalvesystolicannulardimens',
'mvannulusdimensioncmap4c',
'tvannulusdimensionindiastole',
'rvs1tdicmsap4cviewtd',
'rvotdistaljustbelowpulmvlv',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvmidenddiastolicdimension',
'tapsecmap4cviewmmode',
'rvotproximal12oclockdiast',
'age_y_at_echo',
'rvbasalenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'degreeofprbyechoreportif_5',
'rvenddiastolicareacm2ap',
'pslarvotdiastolicdimensionc']

In [21]:
inputs_2a_new_run4 = ['heightonmri',
'tvannulusdimensionindiastole',
'age_y_at_echo',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvbasalenddiastolicdimension',
'tapsecmap4cviewmmode',
'degreeofprbyechoreportif_5',
'rvotproximal12oclockdiast',
'rvlengthenddiastolicdimensio',
'pslarvotdiastolicdimensionc',
'rvenddiastolicareacm2ap']

In [37]:
inputs_2a_new_run5 = ['heightonmri',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvlengthenddiastolicdimensio',
'degreeofprbyechoreportif_5',
'rvbasalenddiastolicdimension',
'rvotproximal12oclockdiast',
'tapsecmap4cviewmmode',
'pslarvotdiastolicdimensionc',
'rvenddiastolicareacm2ap']

In [9]:
inputs_2a_new_run6 = ['heightonmri',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvlengthenddiastolicdimensio',
'rvbasalenddiastolicdimension',
'rvotproximal12oclockdiast',
'pslarvotdiastolicdimensionc',
'rvenddiastolicareacm2ap']

In [15]:
inputs_2a_new_run7 = ['heightonmri',
'tvannulusdimensionindiastole',
'age_y_at_echo',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvbasalenddiastolicdimension',
'tapsecmap4cviewmmode',
'rvotproximal12oclockdiast',
'rvlengthenddiastolicdimensio',
'pslarvotdiastolicdimensionc',
'rvenddiastolicareacm2ap']

In [24]:
inputs_2a_run1 = ['age_y_at_echo',
'degreeofprbyechoreportif_1',
'degreeofprbyechoreportif_2',
'degreeofprbyechoreportif_3',
'degreeofprbyechoreportif_4',
'degreeofprbyechoreportif_5',
'tr_greater_than_mild',
'new_pulm_outflow_col_1',
'new_pulm_outflow_col_2',
'new_pulm_outflow_col_3',
'tapsecmap4cviewmmode',
'rvs1tdicmsap4cviewtd',
'tvangulartiltdegreesap4c',
'mvannulusdimensioncmap4c',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvmidenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvendsystolicareacm2ap4',
'aovalvesystolicannulardimens',
'pslaxpulmvalveannulusindias',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusindiast',
'pssapulmvalveannulusinsysto',
'mpadiastolicflowreversalpres',
'branchpaatleastonediastol',
'antegradediastolicflowateith',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri',
'bsaonmri',
'rv_fac']

In [17]:
inputs_2a_2 = ['age_y_at_echo',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusinsysto',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri']

In [30]:
inputs_2a_run4 = ['age_y_at_echo',
'degreeofprbyechoreportif_5',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusinsysto',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri']

In [34]:
inputs_2a_run5 = ['age_y_at_echo',
'rvenddiastolicareacm2ap',
'rvbasalenddiastolicdimension',
'rvlengthenddiastolicdimensio',
'rvotproximal12oclockdiast',
'rvotdistaljustbelowpulmvlv',
'pssapulmvalveannulusinsysto',
'pslarvotdiastolicdimensionc',
'tvannulusdimensionindiastole',
'weightonmri',
'heightonmri']

In [15]:
# Apply one-hot encoding
df_encoded = pd.get_dummies(merge_df, columns=['pulmonaryoutflowtypeattimeo', 'degreeofprbyechoreportif', 'degreeoftrbyechoreportif'])


In [ ]:
# Apply one-hot encoding - to updated dataset (df_2)
df_encoded_2 = pd.get_dummies(df_2, columns=['new_pulm_outflow_col', 'degreeofprbyechoreportif'])


In [ ]:
df_encoded_2.columns

In [ ]:
df_encoded_2['outcome_rvedvi_hi'].value_counts()

In [ ]:
len(df_encoded_2['mrn'].unique())

In [ ]:
### xgboost auroc curves 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


# setup
X = df_encoded[inputs_2a]
y = df_encoded['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store fpr and tpr for each fold
fprs = []
tprs = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='roc_auc',  # making AUROC our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
plt.figure(figsize=(10, 6))

# do stratification by outcome

scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    auroc_fold_scores = []

    # Fit model
    best_xgb_model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # Calculate AUROC for the current fold
    auroc = roc_auc_score(y_test, y_pred_prob)

    # Add to AUROC, TPR, FPR, and thresholds fold scores
    auroc_fold_scores.append(auroc)

    # Compute ROC curve for the current fold
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Append fpr and tpr to the lists
    fprs.append(fpr)
    tprs.append(tpr)

    # Plot the ROC curve for the current cross-validation fold
    plt.step(fprs[fold], tprs[fold], lw=1, label=f'Fold {fold+1} (AUC = {round(auroc, 3)})')

    # Add to AUROC list
    scores.append(auroc)
    
# calculate average + std AUROC score across all stratified folds
average_auroc = np.mean(scores)
print("Average AUROC Across Stratified Folds:", average_auroc)

std_auroc = np.std(scores)
print("Standard Deviation AUROC Across Stratified Folds:", std_auroc)

# Print AUROC scores for each stratified fold
for i, auroc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUROC: {auroc:.4f}")

# Calculate the mean ROC curve
mean_fpr = np.linspace(0, 1, 100)  # Choose the number of thresholds, here chose 100
tpr_interp = []

for fold in range(len(fprs)):
    tpr_interp.append(np.interp(mean_fpr, fprs[fold], tprs[fold]))

mean_tpr = np.mean(tpr_interp, axis=0)

# plot the average ROC curve and set labels
plt.plot(mean_fpr, mean_tpr, color='b', linestyle='-', lw=2, label=f'Mean ROC (AUC = {round(average_auroc,3)})')

# other plot settings
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Stratified Cross-Validation Folds (Experiment 2A)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
### xgboost auroc with group stratification + nested cv

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

X = df_encoded[inputs_2a_new_run4]
y = df_encoded['outcome_rvedvi_hi']
groups = df_encoded['mrn']  # Group identifier

# Hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Number of splits for cross-validation
n_splits_outer = 5

# GroupKFold for outer cross-validation
outer_cv = GroupKFold(n_splits=n_splits_outer)

# Lists for storing results
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)
tpr_interpolations = []

#save feature importances per fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # RandomizedSearchCV for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='roc_auc',
        cv=3,  # You can adjust this value
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', lw=2, label=f'Mean ROC (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='grey', alpha=0.2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
mean_tpr

In [ ]:
mean_fprs

In [ ]:
# Given values
prevalence = 0.0905
sensitivity = 0.90
specificity = 0.727

# Calculate PPV
ppv = (sensitivity * prevalence) / (sensitivity * prevalence + (1 - specificity) * (1 - prevalence))

# Calculate NPV
npv = (specificity * (1 - prevalence)) / ((1 - sensitivity) * prevalence + specificity * (1 - prevalence))

# Print results
print("Positive Predictive Value (PPV):", ppv)
print("Negative Predictive Value (NPV):", npv)

In [ ]:
df_encoded['outcome_rvedvi_hi'].value_counts()

In [30]:
df_temp = df_encoded[df_encoded['outcome_rvedvi_hi'] == 1]

In [27]:
#save all 2A run 4 stuff
average_performance_2A4 = average_performance
std_dev_performance_2A4 = std_dev_performance
mean_tpr_2A4 = mean_tpr
std_tpr_2A4 = std_tpr
mean_fprs_2A4 = mean_fprs

In [29]:
#save all 2A run 7 stuff
average_performance_2A7 = average_performance
std_dev_performance_2A7 = std_dev_performance
mean_tpr_2A7 = mean_tpr
std_tpr_2A7 = std_tpr
mean_fprs_2A7 = mean_fprs

In [ ]:
#xgboost auroc with nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_encoded_2[inputs_2a_run4]
y = df_encoded_2['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

#create list to store feature importances for each fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=3000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#xgboost auroc with REPEATED (20x) nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# setup
X = df_encoded_2[inputs_2a_run5]
y = df_encoded_2['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5
n_repeats_outer = 20

# make RepeatedStratifiedKFold cross-validation iterator for outer CV
outer_cv = RepeatedStratifiedKFold(n_splits=n_splits_outer, n_repeats=n_repeats_outer, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

#create list to store feature importances for each fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
#plt.figure(figsize=(10, 6))
#plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
#plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('Mean ROC Curve with Standard Deviation')
#plt.legend(loc='lower right')
#plt.show()



In [ ]:
# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')

plt.savefig('2a_run5_auroc_graph.svg', format='svg')
plt.show()

In [72]:
avg_feature_importances_2arun4 = [0.0145127644,
0.03632373,
0.0494950644,
0.0573905092,
0.081441497,
0.09095655,
0.094159966,
0.0947930042,
0.104949613,
0.110177038,
0.1317176888,
0.134082564]

In [ ]:
for f in feature_importances:
    for i in f:
        print(i)
        
    print(" ")
    
    
for i in X_train.columns:
    print(i)

In [79]:
avg_features_1a = [0.227449248,
0.08479753,
0.017493238,
0.262206272,
0.034100496,
0.2407132284,
0.0378129708,
0.035912066,
0.0595149368]

In [84]:
avg_features_3 = [0.078271715,
0.143294996,
0.134644461,
0.360289106,
0.0667977328,
0.216702006]

In [63]:
#get mean and std dev of feature importances 

col0 = []
col1 = []
col2 = []
col3 = []
col4 = []
col5 = []
col6 = []
col7 = []
col8 = []
col9 = []
col10 = []


for f in feature_importances:
    
    count = 0
    for i in f:
        if count == 0:
            col0.append(i)
        if count == 1:
            col1.append(i)
        if count == 2:
            col2.append(i)
        if count == 3:
            col3.append(i)
        if count == 4:
            col4.append(i)
        if count == 5:
            col5.append(i)
        if count == 6:
            col6.append(i)
        if count == 7:
            col7.append(i)
        if count == 8:
            col8.append(i)
        if count == 9:
            col9.append(i)
        if count == 10:
            col10.append(i)
            
        count = count + 1
            
            


In [ ]:
print(np.mean(col10))
print(np.std(col10))

In [73]:
feature_names_2arun4 = ['tvannulusdimensionindiastole',
'heightonmri',
'age_y_at_echo',
'tvangulartiltdegreesap4c',
'pssapulmvalveannulusindiast',
'rvlengthenddiastolicdimensio',
'degreeofprbyechoreportif_5',
'rvbasalenddiastolicdimension',
'rvotproximal12oclockdiast',
'tapsecmap4cviewmmode',
'pslarvotdiastolicdimensionc',
'rvenddiastolicareacm2ap']

In [ ]:
## get feature importance

feature_importance = avg_features_3
feature_names = X_train.columns


# Assuming feature_importance and feature_names are defined as in your code

# Plot feature importance
plt.figure(figsize=(10, 8))

# Set custom tick positions
tick_positions = np.arange(len(feature_importance)) * 1.5  # Adjust the multiplier to increase spacing

plt.barh(tick_positions, feature_importance, tick_label=feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Average Feature Importance Experiment 3')
plt.yticks(tick_positions, feature_names)  # Apply custom tick positions

plt.savefig("avg_feature_performance_3.svg", format='svg')


plt.show()


In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df_encoded[inputs_2a]
y = df_encoded['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 2A)')
plt.show()


In [ ]:
#xgboost pr curves with group strat + nested cv loops

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_encoded[inputs_2a_new_run7]
y = df_encoded['outcome_rvedvi_hi']
groups = df_encoded['mrn']  # Group identifier column

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# Number of splits for outer and inner cross-validation
n_splits_outer = 5
n_splits_inner = 3

# GroupKFold for outer cross-validation
outer_group_kfold = GroupKFold(n_splits=n_splits_outer)

# Lists to store results
precisions_list = []
recalls_list = []
average_aucpr_scores = []

# Outer cross-validation loop
for train_index, test_index in outer_group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    groups_train = groups.iloc[train_index]

    # XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # GroupKFold for inner cross-validation (hyperparameter tuning)
    inner_group_kfold = GroupKFold(n_splits=n_splits_inner)

    # RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='average_precision',
        cv=inner_group_kfold.split(X_train, y_train, groups_train),
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    # PR curve values
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    precisions_list.append(precision)
    recalls_list.append(recall)

    # AUC-PR
    auc_pr = auc(recall, precision)
    average_aucpr_scores.append(auc_pr)

# Calculate and print the average and standard deviation performance across all outer folds
average_aucpr = np.mean(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", average_aucpr)
std_aucpr = np.std(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", std_aucpr)

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))
mean_recall = np.linspace(0, 1, 100)
precision_interp = []

# Interpolate and plot each fold's precision-recall curve
for recall, precision in zip(recalls_list, precisions_list):
    recall_sorted, precision_sorted = zip(*sorted(zip(recall, precision)))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')

plt.subplots_adjust(right=0.75)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds (Experiment 1A)')
plt.show()


In [41]:
#save 2A run 4 stuff

mean_precision_2A4 = mean_precision
mean_recall_2A4 = mean_recall
average_aucpr_2A4 = average_aucpr
std_aucpr_2A4 = std_aucpr
baseline_prevalence_2A4 = baseline_prevalence

In [43]:
mean_precision_2A7 = mean_precision
mean_recall_2A7 = mean_recall
average_aucpr_2A7 = average_aucpr
std_aucpr_2A7 = std_aucpr
baseline_prevalence_2A7 = baseline_prevalence

In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_encoded_2[inputs_2a_run4]
y = df_encoded_2['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=3000,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [ ]:
#pr curves using REPEATED nested cv

from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_encoded_2[inputs_2a_run4]
y = df_encoded_2['outcome_rvedvi_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits and repeats for outer cross-validation
n_splits_outer = 5  # Number of outer splits
n_repeats_outer = 20  # Number of repeats

# make RepeatedStratifiedKFold cross-validation iterator for outer cross-validation
outer_repeated_stratified_kfold = RepeatedStratifiedKFold(n_splits=n_splits_outer, n_repeats=n_repeats_outer, random_state=42)

# Lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

outer_fold_auprcs = []

# Outer Cross-Validation loop
for i, (train_index, test_index) in enumerate(outer_repeated_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # add auc_pr to list
    outer_fold_auprcs.append(auc_pr)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {i+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
average_aucpr = np.mean(outer_fold_auprcs)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(outer_fold_auprcs)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [ ]:
# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()

plt.savefig('2a_run4_auprc_graph.svg', format='svg')
plt.show()

In [15]:
#### Experiment 2B ####

inputs_2b = inputs_2a

df_2b = df_encoded

In [16]:
df_2b = df_2b.dropna(subset=['outcome_rvesv_hi'])

In [ ]:
### xgboost auroc curves 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


# setup
X = df_2b[inputs_2b]
y = df_2b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store fpr and tpr for each fold
fprs = []
tprs = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='roc_auc',  # making AUROC our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
plt.figure(figsize=(10, 6))

# do stratification by outcome

scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    auroc_fold_scores = []

    # Fit model
    best_xgb_model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # Calculate AUROC for the current fold
    auroc = roc_auc_score(y_test, y_pred_prob)

    # Add to AUROC, TPR, FPR, and thresholds fold scores
    auroc_fold_scores.append(auroc)

    # Compute ROC curve for the current fold
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Append fpr and tpr to the lists
    fprs.append(fpr)
    tprs.append(tpr)

    # Plot the ROC curve for the current cross-validation fold
    plt.step(fprs[fold], tprs[fold], lw=1, label=f'Fold {fold+1} (AUC = {round(auroc, 3)})')

    # Add to AUROC list
    scores.append(auroc)
    
# calculate average + std AUROC score across all stratified folds
average_auroc = np.mean(scores)
print("Average AUROC Across Stratified Folds:", average_auroc)

std_auroc = np.std(scores)
print("Standard Deviation AUROC Across Stratified Folds:", std_auroc)

# Print AUROC scores for each stratified fold
for i, auroc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUROC: {auroc:.4f}")

# Calculate the mean ROC curve
mean_fpr = np.linspace(0, 1, 100)  # Choose the number of thresholds, here chose 100
tpr_interp = []

for fold in range(len(fprs)):
    tpr_interp.append(np.interp(mean_fpr, fprs[fold], tprs[fold]))

mean_tpr = np.mean(tpr_interp, axis=0)

# plot the average ROC curve and set labels
plt.plot(mean_fpr, mean_tpr, color='b', linestyle='-', lw=2, label=f'Mean ROC (AUC = {round(average_auroc,3)})')

# other plot settings
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Stratified Cross-Validation Folds (Experiment 2B)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#xgboost auroc with nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_2b[inputs_2b]
y = df_2b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df_2b[inputs_2b]
y = df_2b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 2B)')
plt.show()


In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_2b[inputs_2b]
y = df_2b['outcome_rvesv_hi']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [7]:
##### Experiment 3 #####
inputs_3 = ['tapsecmap4cviewmmode','rvs1tdicmsap4cviewtd',
           'rvenddiastolicareacm2ap', 'rvendsystolicareacm2ap4',
           'bsaonmri', 'rv_fac']

In [8]:
df_3 = merge_df

In [9]:
df_3 = df_3.dropna(subset=['outcome_rvef_low'])

In [ ]:
### xgboost auroc curves 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


# setup
X = df_3[inputs_3]
y = df_3['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store fpr and tpr for each fold
fprs = []
tprs = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='roc_auc',  # making AUROC our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

#store feature importances for each fold
feature_importances = []

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
plt.figure(figsize=(10, 6))

# do stratification by outcome

scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    auroc_fold_scores = []

    # Fit model
    best_xgb_model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # Calculate AUROC for the current fold
    auroc = roc_auc_score(y_test, y_pred_prob)

    # Add to AUROC, TPR, FPR, and thresholds fold scores
    auroc_fold_scores.append(auroc)

    # Compute ROC curve for the current fold
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Append fpr and tpr to the lists
    fprs.append(fpr)
    tprs.append(tpr)

    # Plot the ROC curve for the current cross-validation fold
    plt.step(fprs[fold], tprs[fold], lw=1, label=f'Fold {fold+1} (AUC = {round(auroc, 3)})')

    # Add to AUROC list
    scores.append(auroc)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)
    
# calculate average + std AUROC score across all stratified folds
average_auroc = np.mean(scores)
print("Average AUROC Across Stratified Folds:", average_auroc)

std_auroc = np.std(scores)
print("Standard Deviation AUROC Across Stratified Folds:", std_auroc)

# Print AUROC scores for each stratified fold
for i, auroc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUROC: {auroc:.4f}")

# Calculate the mean ROC curve
mean_fpr = np.linspace(0, 1, 100)  # Choose the number of thresholds, here chose 100
tpr_interp = []

for fold in range(len(fprs)):
    tpr_interp.append(np.interp(mean_fpr, fprs[fold], tprs[fold]))

mean_tpr = np.mean(tpr_interp, axis=0)

# plot the average ROC curve and set labels
plt.plot(mean_fpr, mean_tpr, color='b', linestyle='-', lw=2, label=f'Mean ROC (AUC = {round(average_auroc,3)})')

# other plot settings
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Stratified Cross-Validation Folds (Experiment 3)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
### xgboost auroc with group stratification and nested cv

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

X = df_3[inputs_3]
y = df_3['outcome_rvef_low']
groups = df_3['mrn']  # Group identifier

# Hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Number of splits for cross-validation
n_splits_outer = 5

# GroupKFold for outer cross-validation
outer_cv = GroupKFold(n_splits=n_splits_outer)

# Lists for storing results
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)
tpr_interpolations = []

#store feature importances per fold
feature_importances = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # RandomizedSearchCV for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='roc_auc',
        cv=3,  # You can adjust this value
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0
    tpr_interpolations.append(tpr_interp)
    
    #get feature importance for fold
    fi = best_model.feature_importances_
    sum_fi = sum(fi)
    temp_fi = [x / sum_fi for x in fi]
    feature_importances.append(fi)

# Average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
#mean_tpr[-1] = 1.0
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves
plt.figure(figsize=(7, 7))
plt.plot(mean_fprs, mean_tpr, color='b', lw=2, label=f'Mean ROC (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='gray', alpha=0.15)

plt.plot([], [], color='grey', alpha=0.2, linewidth=10, label='±1 SD')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.savefig("roc_curve_with_std_3.svg", format='svg')

plt.show()


In [11]:
mean_fprs3 = mean_fprs
mean_tpr3 = mean_tpr
average_performance3 = average_performance
std_tpr3 = std_tpr

In [ ]:
# Plotting the ROC curves - overlayed plot with exp 4
plt.figure(figsize=(7, 7))
plt.plot(mean_fprs3, mean_tpr3, color='b', lw=2, label=f'Mean ROC for ASE Guidelines (AUC = {average_performance3:.2f})')
plt.fill_between(mean_fprs3, mean_tpr3 - std_tpr3, mean_tpr3 + std_tpr3, color='gray', alpha=0.15)

plt.plot(mean_fprs, mean_tpr, color='g', lw=2, label=f'Mean ROC for Expanded Dataset (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='gray', alpha=0.15)

plt.plot([], [], color='grey', alpha=0.2, linewidth=10, label='±1 SD')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.savefig("roc_curve_rvef_overlayed.svg", format='svg')

plt.show()

In [ ]:
mean_recall3 = mean_recall
mean_precision4 = mean_precision
average_aucpr4 = average_aucpr
baseline_prevalence4 = baseline_prevalence

In [ ]:
#xgboost auroc with nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_3[inputs_3]
y = df_3['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
## get feature importance

feature_importance = best_xgb_model.feature_importances_
feature_names = X_train.columns
# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, tick_label=feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('XGBoost Classifier Feature Importance')
plt.show()

In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df_3[inputs_3]
y = df_3['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 3)')
plt.show()


In [ ]:
#xgboost pr curves with group strat + nested cv loops

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_3[inputs_3]
y = df_3['outcome_rvef_low']
groups = df_3['mrn']  # Group identifier column

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# Number of splits for outer and inner cross-validation
n_splits_outer = 5
n_splits_inner = 3

# GroupKFold for outer cross-validation
outer_group_kfold = GroupKFold(n_splits=n_splits_outer)

# Lists to store results
precisions_list = []
recalls_list = []
average_aucpr_scores = []

# Outer cross-validation loop
for train_index, test_index in outer_group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    groups_train = groups.iloc[train_index]

    # XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # GroupKFold for inner cross-validation (hyperparameter tuning)
    inner_group_kfold = GroupKFold(n_splits=n_splits_inner)

    # RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='average_precision',
        cv=inner_group_kfold.split(X_train, y_train, groups_train),
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    # PR curve values
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    precisions_list.append(precision)
    recalls_list.append(recall)

    # AUC-PR
    auc_pr = auc(recall, precision)
    average_aucpr_scores.append(auc_pr)

# Calculate and print the average and standard deviation performance across all outer folds
average_aucpr = np.mean(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", average_aucpr)
std_aucpr = np.std(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", std_aucpr)

# Plotting
fig, ax = plt.subplots(figsize=(7, 7))
mean_recall = np.linspace(0, 1, 100)
precision_interp = []

# Interpolate and plot each fold's precision-recall curve
for recall, precision in zip(recalls_list, precisions_list):
    recall_sorted, precision_sorted = zip(*sorted(zip(recall, precision)))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')

ax.fill_between(mean_recall, mean_precision - std_aucpr, mean_precision + std_aucpr, color='gray', alpha=0.15)


plt.subplots_adjust(right=0.75)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds (Experiment 3)')
plt.savefig("pr_curve_with_std_3.svg", format='svg')

plt.show()


In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(7, 7))
mean_recall = np.linspace(0, 1, 100)
precision_interp = []

# Interpolate and plot each fold's precision-recall curve
for recall, precision in zip(recalls_list, precisions_list):
    recall_sorted, precision_sorted = zip(*sorted(zip(recall, precision)))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')

ax.fill_between(mean_recall, mean_precision - std_aucpr, mean_precision + std_aucpr, color='gray', alpha=0.15)
plt.ylim([0, 1])  # Setting y-axis limits from 0 to 1


plt.subplots_adjust(right=0.75)
plt.legend(loc='best', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds (Experiment 3)')
plt.savefig("pr_curve_with_std_3.svg", format='svg')

plt.show()

In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_3[inputs_3]
y = df_3['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()


In [16]:
#### Experiment 4 #####

inputs_4 = inputs_2a

In [17]:
df_4 = df_encoded

In [18]:
df_4 = df_4.dropna(subset=['outcome_rvef_low'])

In [ ]:
### xgboost auroc curves 

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


# setup
X = df_4[inputs_4]
y = df_4['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store fpr and tpr for each fold
fprs = []
tprs = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='roc_auc',  # making AUROC our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
plt.figure(figsize=(10, 6))

# do stratification by outcome

scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    auroc_fold_scores = []

    # Fit model
    best_xgb_model.fit(X_train, y_train)

    # Predict probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # Calculate AUROC for the current fold
    auroc = roc_auc_score(y_test, y_pred_prob)

    # Add to AUROC, TPR, FPR, and thresholds fold scores
    auroc_fold_scores.append(auroc)

    # Compute ROC curve for the current fold
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

    # Append fpr and tpr to the lists
    fprs.append(fpr)
    tprs.append(tpr)

    # Plot the ROC curve for the current cross-validation fold
    plt.step(fprs[fold], tprs[fold], lw=1, label=f'Fold {fold+1} (AUC = {round(auroc, 3)})')

    # Add to AUROC list
    scores.append(auroc)
    
# calculate average + std AUROC score across all stratified folds
average_auroc = np.mean(scores)
print("Average AUROC Across Stratified Folds:", average_auroc)

std_auroc = np.std(scores)
print("Standard Deviation AUROC Across Stratified Folds:", std_auroc)

# Print AUROC scores for each stratified fold
for i, auroc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUROC: {auroc:.4f}")

# Calculate the mean ROC curve
mean_fpr = np.linspace(0, 1, 100)  # Choose the number of thresholds, here chose 100
tpr_interp = []

for fold in range(len(fprs)):
    tpr_interp.append(np.interp(mean_fpr, fprs[fold], tprs[fold]))

mean_tpr = np.mean(tpr_interp, axis=0)

# plot the average ROC curve and set labels
plt.plot(mean_fpr, mean_tpr, color='b', linestyle='-', lw=2, label=f'Mean ROC (AUC = {round(average_auroc,3)})')

# other plot settings
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Stratified Cross-Validation Folds (Experiment 4)')
plt.legend(loc='lower right')
plt.show()


In [ ]:
### xgboost auroc with group stratification and nested cv

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

X = df_4[inputs_4]
y = df_4['outcome_rvef_low']
groups = df_4['mrn']  # Group identifier

# Hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}

# XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Number of splits for cross-validation
n_splits_outer = 5

# GroupKFold for outer cross-validation
outer_cv = GroupKFold(n_splits=n_splits_outer)

# Lists for storing results
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)
tpr_interpolations = []

y_train_vals = []
y_test_vals = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # RandomizedSearchCV for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='roc_auc',
        cv=3,  # You can adjust this value
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0
    tpr_interpolations.append(tpr_interp)
    
    y_train_vals.append(y_train)
    y_test_vals.append(y_test)

# Average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves
plt.figure(figsize=(7, 7))
plt.plot(mean_fprs, mean_tpr, color='g', lw=2, label=f'Mean ROC (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='grey', alpha=0.2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')

plt.savefig("roc_curve_4.svg", format='svg')

plt.show()


In [ ]:
# Plotting the ROC curves
plt.figure(figsize=(7, 7))
plt.plot(mean_fprs, mean_tpr, color='g', lw=2, label=f'Mean ROC (AUC = {average_performance:.2f})')
plt.fill_between(mean_fprs, mean_tpr - std_tpr, mean_tpr + std_tpr, color='grey', alpha=0.2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')

# Add line representing chance
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='black', label='Chance')


plt.savefig("roc_curve_4.svg", format='svg')

plt.show()

In [ ]:
#y_train_vals[1]

temp=0
for i in y_test_vals[4]:
    temp += i

print(temp)

In [ ]:
#xgboost auroc with nested cross validation

import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_4[inputs_4]
y = df_4['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [2, 3],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'reg_alpha': [0.0, 1.0, 10.0],
    'reg_lambda': [0.0, 1.0, 10.0],
    'gamma': [0, 1.0, 5.0],
    'min_child_weight': [1, 10, 20]
}


# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Define the number of splits for both inner and outer cross-validation
n_splits_inner = 3
n_splits_outer = 5

# make StratifiedKFold cross-validation iterator for outer CV
outer_cv = StratifiedKFold(n_splits=n_splits_outer, shuffle=True, random_state=42)

# Lists to store results of each outer fold
outer_fold_results = []
mean_fprs = np.linspace(0, 1, 100)  # Define evenly spaced FPR values for interpolation
tpr_interpolations = []

# Outer Cross-Validation loop
for train_index, test_index in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # make StratifiedKFold cross-validation iterator for inner CV (hyperparameter tuning)
    inner_cv = StratifiedKFold(n_splits=n_splits_inner, shuffle=True, random_state=42)

    # make RandomizedSearchCV object for hyperparameter tuning within each outer fold
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=1000,
        scoring='roc_auc',
        cv=inner_cv,
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Perform hyperparameter tuning on the training data of the outer fold
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred_prob)
    outer_fold_results.append(auroc)

    # ROC curve for the outer fold
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    tpr_interp = np.interp(mean_fprs, fpr, tpr)
    tpr_interp[0] = 0.0  # Ensure starting at 0
    tpr_interpolations.append(tpr_interp)

# Calculate and print the average performance across all outer folds
average_performance = np.mean(outer_fold_results)
std_dev_performance = np.std(outer_fold_results)
print("Average AUROC Across Outer Folds:", average_performance)
print("Standard Deviation of AUROC Across Outer Folds:", std_dev_performance)

# Compute mean and standard deviation of the interpolated TPR
mean_tpr = np.mean(tpr_interpolations, axis=0)
mean_tpr[-1] = 1.0  # Ensure ending at 1
std_tpr = np.std(tpr_interpolations, axis=0)

# Plotting the ROC curves with shading for standard deviation
plt.figure(figsize=(10, 6))
plt.plot(mean_fprs, mean_tpr, color='b', label=f'Mean ROC (AUC = {average_performance:.2f})', lw=2)
plt.fill_between(mean_fprs, mean_tpr-std_tpr, mean_tpr+std_tpr, color='grey', alpha=0.2, label='±1 std. dev.')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Mean ROC Curve with Standard Deviation')
plt.legend(loc='lower right')
plt.show()


In [ ]:
## get feature importance

feature_importance = best_xgb_model.feature_importances_
feature_names = X_train.columns
# Plot feature importance
plt.figure(figsize=(10, 8))

# Set custom tick positions
tick_positions = np.arange(len(feature_importance)) * 1.5  # Adjust the multiplier to increase spacing

plt.barh(tick_positions, feature_importance, tick_label=feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('XGBoost Classifier Feature Importance')
plt.yticks(tick_positions, feature_names)  # Apply custom tick positions

plt.show()

In [ ]:
########## xgboost PR curves


from sklearn.metrics import precision_recall_curve, auc

# setup
X = df_4[inputs_4]
y = df_4['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make XGBoost model
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# make StratifiedKFold cross-validation iterator for outcome stratification
n_splits = 5  # Number of splits
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# make RandomizedSearchCV object
random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_grid,
    n_iter=1000,  # chose the number of random parameter combinations to try
    scoring='average_precision',  # making avg precision our scoring metric
    cv=stratified_kfold.split(X, y),  # Use stratified folds for outcome stratification
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# fit RandomizedSearchCV object on our data for hyperparameter tuning
random_search.fit(X, y)

# save best hyperparameters and best model from the random search
best_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

# print best hyperparameters
print("Best Hyperparameters:", best_params)

# make plot for figure outside of loop to be able to overlay all plots at the end
#plt.figure(figsize=(8, 6))
fig, ax = plt.subplots(figsize=(8, 6))


# do stratification by outcome
scores = []
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # fit model
    best_xgb_model.fit(X_train, y_train)

    #  probabilities
    y_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]

    # do PR curve vals
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

    # store precision and recall per fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # plot the PR curve for the current cross-validation fold
    ax.plot(recalls_list[fold], precisions_list[fold], lw=1, label=f'Fold {fold+1} (AUCPR = {round(auc_pr, 3)})')

    # Add AUC-PR to the scores list
    scores.append(auc_pr)

# calculate average AUC-PR score across all stratified folds
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Folds:", std_aucpr)

# Print AUROC scores for each stratified fold
for i, auprc in enumerate(scores):
    print(f"Stratified Fold {i+1} AUPRC: {auprc:.4f}")

baseline_prevalence = y.mean()



# calculate the mean PR curve - must do sorting and interpolation to fit data correctly
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# plot  mean PR curve
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# plot the chance PR curve (should be a diagonal line from 0,1 to 1,0)
#plt.plot([0, 1], [1, 0], color='gray', linestyle='--', lw=2, label='Chance')

# Plot baseline prevalence as a horizontal line
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')


# Adjust layout to make room for the legend
plt.subplots_adjust(right=0.75)

# Place the legend outside of the plot
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))

# Other plot settings
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('AUPRC Curves for Stratified Cross-Validation Folds (Experiment 4)')
plt.show()


In [ ]:
#xgboost pr curves with group strat + nested cv loops

import numpy as np
import xgboost as xgb
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# setup
X = df_4[inputs_4]
y = df_4['outcome_rvef_low']
groups = df_4['mrn']  # Group identifier column

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# Number of splits for outer and inner cross-validation
n_splits_outer = 5
n_splits_inner = 3

# GroupKFold for outer cross-validation
outer_group_kfold = GroupKFold(n_splits=n_splits_outer)

# Lists to store results
precisions_list = []
recalls_list = []
average_aucpr_scores = []

# Outer cross-validation loop
for train_index, test_index in outer_group_kfold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    groups_train = groups.iloc[train_index]

    # XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # GroupKFold for inner cross-validation (hyperparameter tuning)
    inner_group_kfold = GroupKFold(n_splits=n_splits_inner)

    # RandomizedSearchCV for hyperparameter tuning
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,
        scoring='average_precision',
        cv=inner_group_kfold.split(X_train, y_train, groups_train),
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    # Hyperparameter tuning
    random_search.fit(X_train, y_train)

    # Best model for this outer fold
    best_model = random_search.best_estimator_

    # Evaluate the best model on the test set of the outer fold
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]

    # PR curve values
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    precisions_list.append(precision)
    recalls_list.append(recall)

    # AUC-PR
    auc_pr = auc(recall, precision)
    average_aucpr_scores.append(auc_pr)

# Calculate and print the average and standard deviation performance across all outer folds
average_aucpr = np.mean(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", average_aucpr)
std_aucpr = np.std(average_aucpr_scores)
print("Average AUPRC Across Group-Stratified Folds:", std_aucpr)

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))
mean_recall = np.linspace(0, 1, 100)
precision_interp = []

# Interpolate and plot each fold's precision-recall curve
for recall, precision in zip(recalls_list, precisions_list):
    recall_sorted, precision_sorted = zip(*sorted(zip(recall, precision)))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)
ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')

plt.subplots_adjust(right=0.75)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curves for Group-Stratified Cross-Validation Folds (Experiment 4)')

plt.savefig('pr_curve_exp_4.svg')
plt.show()


In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC for ASE Guidelines Model (AUC = {round(average_aucpr, 3)})')
ax.plot(mean_recall4, mean_precision4, color='g', linestyle='-', lw=2, label=f'Mean AUPRC for Expanded Dataset Model (AUC = {round(average_aucpr4, 3)})')



baseline_prevalence = y.mean()
ax.hlines(baseline_prevalence, 0, 1, colors='red', linestyles='dashed', lw=2, label=f'Baseline Prevalence ({round(baseline_prevalence, 3)})')



plt.subplots_adjust(right=0.75)
plt.legend(loc='upper right')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Prediction of RVEF < 47%: Precision-Recall Curves')

plt.savefig('pr_curves_rvef_overlayed.svg')
plt.savefig('pr_curves_rvef_overlayed.png')

plt.show()

In [28]:
mean_recall4 = mean_recall
mean_precision4 = mean_precision
average_aucpr4 = average_aucpr
baseline_prevalence4 = baseline_prevalence

In [ ]:
#pr curves using nested cv

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

# setup
X = df_4[inputs_4]
y = df_4['outcome_rvef_low']

# hyperparameter grid
param_grid = {
    'max_depth': [1, 2, 3],
    'learning_rate': [0.1],
    'n_estimators': [10, 25, 50, 100, 200],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0, 20.0],
    'gamma': [0, 0.1, 0.5, 1.0, 5.0, 10, 20],
    'min_child_weight': [0.1, 0.5, 1, 5, 10, 20, 25]
}

# make StratifiedKFold cross-validation iterator for outer cross-validation
n_outer_splits = 5  # Number of outer splits
outer_stratified_kfold = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)

# Add empty lists to store precision and recall for each fold
precisions_list = []
recalls_list = []

# Outer loop: iterate over outer folds
for outer_fold, (train_index, test_index) in enumerate(outer_stratified_kfold.split(X, y)):
    X_train_outer, X_test_outer = X.iloc[train_index], X.iloc[test_index]
    y_train_outer, y_test_outer = y.iloc[train_index], y.iloc[test_index]

    # make XGBoost model
    xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

    # make StratifiedKFold cross-validation iterator for inner cross-validation
    n_inner_splits = 3  # Number of inner splits
    inner_stratified_kfold = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

    # make RandomizedSearchCV object
    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=500,  # chose the number of random parameter combinations to try
        scoring='average_precision',  # making avg precision our scoring metric
        cv=inner_stratified_kfold.split(X_train_outer, y_train_outer),  # Use stratified folds for outcome stratification
        verbose=2,
        n_jobs=-1
    )

    # fit RandomizedSearchCV object on training data for hyperparameter tuning
    random_search.fit(X_train_outer, y_train_outer)

    # save best hyperparameters and best model from the random search
    best_params = random_search.best_params_
    best_xgb_model = random_search.best_estimator_

    # fit best model on the entire training set from the outer fold
    best_xgb_model.fit(X_train_outer, y_train_outer)

    # predict probabilities on the test set from the outer fold
    y_pred_prob = best_xgb_model.predict_proba(X_test_outer)[:, 1]

    # compute PR curve values
    precision, recall, _ = precision_recall_curve(y_test_outer, y_pred_prob)

    # store precision and recall per outer fold
    precisions_list.append(precision)
    recalls_list.append(recall)

    # calculate Area Under the Precision-Recall Curve (AUC-PR)
    auc_pr = auc(recall, precision)

    # print AUC-PR for the outer fold
    print(f"Outer Fold {outer_fold+1} AUC-PR: {auc_pr:.4f}")

# calculate the mean PR curve
mean_recall = np.linspace(0, 1, 100)  # choose number of thresholds, here chose 100
precision_interp = []

for fold in range(len(recalls_list)):
    # must sort recalls first before interpolation
    recall_sorted, precision_sorted = zip(*sorted(zip(recalls_list[fold], precisions_list[fold])))
    precision_interp.append(np.interp(mean_recall, recall_sorted, precision_sorted))

mean_precision = np.mean(precision_interp, axis=0)

# calculate average AUC-PR score across all outer folds
scores = [auc(recalls_list[i], precisions_list[i]) for i in range(len(recalls_list))]
average_aucpr = np.mean(scores)
print("Average AUPRC Across Stratified Outer Folds:", average_aucpr)

std_aucpr = np.std(scores)
print("Standard Deviation AUPRC Across Stratified Outer Folds:", std_aucpr)

# plot mean PR curve
plt.figure(figsize=(8, 6))
plt.plot(mean_recall, mean_precision, color='b', linestyle='-', lw=2, label=f'Mean AUPRC (AUC = {round(average_aucpr, 3)})')

# Other plot settings
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Curve for Nested Cross-Validation with RandomizedSearchCV')
plt.legend()
plt.show()
